### HW04: Practice with feature engineering, splitting data, and fitting and regularizing linear models

Anais Corona Perez (NetID: coronaperez)

### Hello Students:

- Start by downloading HW04.ipynb from this folder. Then develop it into your solution.
- Write code where you see "... your code here ..." below.
  (You are welcome to use more than one cell.)
- If you have questions, please ask them in class, office hours, or piazza. Our TA
  and I are very happy to help with the programming (provided you start early
  enough, and provided we are not helping so much that we undermine your learning).
- When you are done, run these Notebook commands:
  - Shift-L (once, so that line numbers are visible)
  - Kernel > Restart and Run All (run all cells from scratch)
  - Esc S (save)
  - File > Download as > HTML
- Turn in:
  - HW04.ipynb to Canvas's HW04.ipynb assignment
  - HW04.html to Canvas's HW04.html assignment
  - As a check, download your files from Canvas to a new 'junk' folder. Try 'Kernel > Restart
  and Run All' on the '.ipynb' file to make sure it works. Glance through the '.html' file.
- Turn in partial solutions to Canvas before the deadline. e.g. Turn in part 1,
  then parts 1 and 2, then your whole solution. That way we can award partial credit
  even if you miss the deadline. We will grade your last submission before the deadline.

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

## 1. Feature engineering (one-hot encoding and data imputation)

(Note: This paragraph is not instructions but rather is to communicate context for this exercise. We use the same Titanic data we used in HW02:
- There we used `df.dropna()` to drop any observations with missing values; here we use data imputation instead.
- There we manually did one-hot encoding of the categorical `Sex` column by making a `Female` column; here we do the same one-hot encoding with the help of pandas's `df.join(pd.get_dummies())`.
- There we used a decision tree; here we use $k$-NN.

We evaluate how these strategies can improve model performance by allowing us to use columns with categorical or missing data.)

### 1a. Read the data from [http://www.stat.wisc.edu/~jgillett/451/data/kaggle_titanic_train.csv](http://www.stat.wisc.edu/~jgillett/451/data/kaggle_titanic_train.csv).
- Retain only these columns: Survived, Pclass, Sex, Age, SibSp, Parch.
- Display the first 7 rows.

These data are described at [https://www.kaggle.com/competitions/titanic/data](https://www.kaggle.com/competitions/titanic/data) (click on the small down-arrow to see the "Data Dictionary"), which is where they are from.
- Read that "Data Dictionary" paragraph (with your eyes, not python) so you understand what each column represents.


In [2]:
df = pd.read_csv('http://www.stat.wisc.edu/~jgillett/451/data/kaggle_titanic_train.csv')[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch']]
df.head(7)

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,male,22.0,1,0
1,1,1,female,38.0,1,0
2,1,3,female,26.0,0,0
3,1,1,female,35.0,1,0
4,0,3,male,35.0,0,0
5,0,3,male,NaN,0,0
6,0,1,male,54.0,0,0


### 1b. Try to train a $k$NN model to predict $y=$ 'Survived' from $X=$ these features: 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch'.
- Use $k = 3$ and the (default) euclidean metric.
- Notice at the bottom of the error message that it fails with the error "ValueError: could not convert string to float: 'male'".
- Comment out your .fit() line so the cell can run without error.

In [3]:
knn = KNeighborsClassifier(n_neighbors = 3, metric = 'euclidean')
#knn.fit(df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch']], df.Survived)

### 1c. Try to train again, this time without the 'Sex' feature.
- Notice that it fails because "Input contains NaN".
- Comment out your .fit() line so the cell can run without error.
- Run `X.isna().any()` (where X is the name of your DataFrame of features) to see that
  the 'Age' feature has missing values. (You can see the first missing value in
  the sixth row that you displayed above.)

In [4]:
knn = KNeighborsClassifier(n_neighbors = 3)
#knn.fit(df[['Pclass', 'Age', 'SibSp', 'Parch']], df.Survived)
df[['Pclass', 'Age', 'SibSp', 'Parch']].isna().any()

Pclass    False
Age        True
SibSp     False
Parch     False
dtype: bool

### 1d. Train without the 'Sex' and 'Age' features.
- Report accuracy on the training data with a line of the form
  `Accuracy on training data is  0.500` (0.500 may not be correct).

In [5]:
knn.fit(df[['Pclass', 'SibSp', 'Parch']], df.Survived)
print(f'Accuracy on training data is:', round(knn.score(df[['Pclass', 'SibSp', 'Parch']], df.Survived), 3))


Accuracy on training data is: 0.633


### 1e.  Use one-hot encoding
to include a binary 'male'  feature made from the 'Sex' feature. (Or include a binary 'female'
feature, according to your preference. Using both is unnecessary since either is the logical
negation of the other.) That is, train on these features: 'Pclass', 'SibSp', 'Parch', 'male'.
- Use pandas's df.join(pd.get_dummies())`.
- Report training accuracy as before.

In [6]:
df = df.join(pd.get_dummies(df['Sex']))
knn.fit(df[['Pclass', 'SibSp', 'Parch', 'male']], df.Survived)
print(f'Accuracy on training data is:', round(knn.score(df[['Pclass', 'SibSp', 'Parch', 'male']], df.Survived), 3))

Accuracy on training data is: 0.744


### 1f. Use data imputation
to include an 'age' feature made from 'Age' but replacing each missing value with the median
of the non-missing ages. That is, train on these features: 'Pclass', 'SibSp', 'Parch', 'male',
'age'.

- Report training accuracy as before.

In [7]:
# Data imputation and replacement
imp = SimpleImputer(missing_values = np.nan, strategy = 'median', fill_value = None).fit_transform(df[['Age']])
df['Age'] = imp

#Train
knn.fit(df[['Pclass', 'Age', 'SibSp', 'Parch', 'male']], df.Survived)
print(f'Accuracy on training data is:', round(knn.score(df[['Pclass','Age', 'SibSp','Parch', 'male']], df.Survived), 3))




Accuracy on training data is: 0.861


## 2. Explore model fit, overfit, and regularization in the context of multiple linear regression

### 2a. Prepare the data:
- Read [http://www.stat.wisc.edu/~jgillett/451/data/mtcars.csv](http://www.stat.wisc.edu/~jgillett/451/data/mtcars.csv) into a DataFrame.
- Set a variable `X` to the subset consisting of all columns except `mpg`.
- Set a variable `y` to the `mpg` column.
- Use `train_test_split()` to split `X` and `y` into `X_train`, `X_test`, `y_train`, and `y_test`.
  - Reserve half the data for training and half for testing.
  - Use `random_state=0` to get reproducible results.

In [8]:
df = pd.read_csv('http://www.stat.wisc.edu/~jgillett/451/data/mtcars.csv')
X = df.loc[:,df.columns != 'mpg'].drop(['Unnamed: 0'], axis = 1)
y = df.mpg

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state = 0)

### 2b. Train three models on the training data and evaluate each on the test data:
- `LinearRegression()`
- `Lasso()`
- `Ridge()`

The evaluation consists in displaying MSE$_\text{train}, $ MSE$_\text{test}$, and the coefficients $\mathbf{w}$ for each model.

In [9]:
# Train on training data:
linear = linear_model.LinearRegression().fit(X_train, y_train)
lasso = linear_model.Lasso().fit(X_train, y_train)
ridge = linear_model.Ridge().fit(X_train, y_train)

#Obtain MSEs: 
lin_error_train = mean_squared_error(linear.predict(X_train), y_train)
lin_error_test = mean_squared_error(linear.predict(X_test), y_test)

lasso_error_train = mean_squared_error(lasso.predict(X_train), y_train)
lasso_error_test = mean_squared_error(lasso.predict(X_test), y_test)

ridge_error_train = mean_squared_error(ridge.predict(X_train), y_train)
ridge_error_test = mean_squared_error(ridge.predict(X_test), y_test)

#Print results
print(f'Evaluation of Linear model: \n MSE train: {lin_error_train:.3} \n MSE test: {lin_error_test:.3} \n Intercept: {linear.intercept_:.3} \n Coefficients: {linear.coef_}\n')
print(f'Evaluation of Lasso model: \n MSE train: {lasso_error_train:.3} \n MSE test: {lasso_error_test:.3} \n Intercept: {lasso.intercept_:.3} \n Coefficients: {lasso.coef_}\n')
print(f'Evaluation of Ridge model: \n MSE train: {ridge_error_train:.3} \n MSE test: {ridge_error_test:.3} \n Intercept: {ridge.intercept_:.3} \n Coefficients: {ridge.coef_}\n')

Evaluation of Linear model: 
 MSE train: 0.386 
 MSE test: 30.2 
 Intercept: -70.3 
 Coefficients: [ 1.08241627  0.02500827  0.02593759  3.12997296 -7.34326296  3.93233873
 -4.08551838 -1.2164054   5.98718293 -0.77097967]

Evaluation of Lasso model: 
 MSE train: 5.69 
 MSE test: 13.0 
 Intercept: 33.8 
 Coefficients: [-0.         -0.03718773 -0.01681757  0.         -0.          0.
  0.          0.          0.         -0.84712891]

Evaluation of Ridge model: 
 MSE train: 1.99 
 MSE test: 11.2 
 Intercept: 7.25 
 Coefficients: [-0.01392446 -0.00744628  0.00400324  0.79271685 -3.21854592  1.09825372
 -0.48236992  0.47079795  1.49546846 -1.04547895]



### 2c. Answer a few questions about the models:
- Which one best fits the training data?
- Which one best fits the test data?
- Which one does feature selection by setting most coefficients to zero?- 

The linear regression model best fits the training data as it has the smallest $MSE_{train}$ result of all the models. Similarly, the model that best fits the test data is the ridge model for having the smallest $MSE_{test}$ result. The lasso model does feature selection as it sets most coefficients to zero.

In [7]:
import math
(3/4)*(-(1/3)*math.log2(1/3)-(2/3)*math.log2(2/3))


0.6887218755408672

In [10]:
1/2-(3/4)*math.log2(3/4)

0.8112781244591328